In [1]:
import os
import datetime
import json
import srt
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled

playlist_id = "PLbtegW3d3L4IpwMvfSW8ceoeVoP5p2iiH"
playlist_url = f"https://youtube.com/playlist?list={playlist_id}"
youtube_dl_cmd = f'youtube-dl -j --flat-playlist "{playlist_url}"'
json_metas = os.popen(youtube_dl_cmd).readlines()

for json_meta in json_metas:
    meta = json.loads(json_meta)
    url = meta['url']
    title = meta['title']
    output_path = f"subs/{title.replace('/', '.')}.srt"

    if os.path.exists(output_path):
        print(f"skipping already downloaded {url}")
        continue

    try:
        sub = YouTubeTranscriptApi.get_transcript(url)
        print(f"subtitles are available for {url}")
        
        sub_frames = []
        for i, s in enumerate(sub):
            sub_frames.append(srt.Subtitle(
                index=i+1,
                start=datetime.timedelta(s['start']),
                end=datetime.timedelta(s['start']+s['duration']),
                content=s['text']
            ))
        sub_srt = srt.compose(sub_frames)
        
        with open(output_path, "w") as sub_file:
            sub_file.write(sub_srt)
            
    except TranscriptsDisabled:
        print(f"subtitles are missing for {url}")
        continue